In [ ]:
from google.colab import files

# Upload the file from your local system
uploaded = files.upload()


Saving names.txt to names.txt


In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(words[:10])
len(words)

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


32033

In [ ]:
# mappings of characters to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}


In [ ]:
N = torch.zeros((27, 27, 27), dtype=torch.int32)

In [ ]:

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    N[ix1, ix2, ix3] += 1

In [ ]:
#Counting Method
g = torch.Generator().manual_seed(2147483647)
N_bigram = N.view(27*27, 27)
P = (N_bigram + 1).float()
print(P.sum(dim=1, keepdim=True).shape)
print(P[54])
P = P / P.sum(dim=1, keepdim=True)
#P[torch.isnan(P)] = 1.0 / 27
print(P[54])
#print(row_sums)
print (P[364])
for i in range(50):
  out = []
  ix1 = 0
  ix2 = 0
  while True:
    #px = ix % 27
    p = P[ix1 * 27 + ix2]
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
         break
    ix1 = ix2
    ix2 = ix
  print(''.join(out))





torch.Size([729, 1])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370])
tensor([0.0154, 0.3744, 0.0051, 0.0051, 0.0051, 0.1744, 0.0051, 0.0051, 0.0051,
        0.1897, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0308, 0.0051, 0.0051,
        0.0154, 0.0051, 0.0051, 0.0103, 0.0051, 0.0051, 0.0051, 0.0923, 0.0051])
junide.
ilyasid.
prelay.
ocin.
fairritoper.
sathen.
dannaaryanileniassibduinrwin.
lessiyanayla.
te.
farmumthyfortumj.
ponn.
zena.
jaylicore.
ya.
zoffra.
jamilyn.
fmouis.
yah.
wanaasnhavi.
honszxhddion.
mathani.
zie.
paun.
ty.
tin.
rosli.
ish.
ubri.
mjeaurickivina.
ha.
van.
cra.
raydnh.
fraita.
malyn.
brey.
ugi.
zavarocbzthemiraya.
ath.
.
.
nely.
favisotten.
salee.
grin.

In [ ]:
# loss in counting
log_likelihood = 0.0
n = 0
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    prob = P[ix1 * 27 + ix2, ix3]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
print(f'{log_likelihood=}')
nil = -log_likelihood
print(f'{nil=}')
print(n)
print(f'{nil/n}')

log_likelihood=tensor(-410414.9688)
nil=tensor(410414.9688)
196113
2.092747449874878


In [ ]:
# neural net method
import random
random.seed(42)
random.shuffle(words)
num_words = len(words)
train_size = int(0.8 * num_words)
dev_size = int(0.1 * num_words)
train_words = words[:train_size]
dev_words = words[train_size:train_size+dev_size]
test_words = words[train_size+dev_size:]

#function for data preparation
def data_prep(words, stoi):
  xs, ys = [], []
  for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
      ix1 = stoi[ch1]
      ix2 = stoi[ch2]
      ix3 = stoi[ch3]
      xs.append(ix1 *27 + ix2)
      ys.append(ix3)
  return torch.tensor(xs), torch.tensor(ys)
train_xs, train_ys = data_prep(train_words, stoi)
dev_xs, dev_ys = data_prep(dev_words, stoi)
test_xs, test_ys = data_prep(test_words, stoi)
full_set_xs, full_set_ys = data_prep(words, stoi)
g = torch.Generator().manual_seed(2147483647)
intial_weights = torch.randn((27 * 27, 27), generator=g, requires_grad=True)

def train_model_one_hot_softmax(xs, ys, w, reg_strength, epochs, lr):
    w = intial_weights.clone().detach().requires_grad_(True)
    for epoch in range(epochs):
        # Forward pass
        xenc = F.one_hot(xs, num_classes=27*27).float()
        logits = xenc @ w
        counts = logits.exp()
        probs = counts / counts.sum(1, keepdims=True)
        loss = -probs[torch.arange(len(ys)), ys].log().mean()
        # Regularization
        reg_loss = reg_strength * (w**2).mean()
        total_loss = loss + reg_loss

        # Backward pass
        w.grad = None  # set to 0 the gradient
        total_loss.backward()

        # Update
        w.data += -lr * w.grad
    return w,total_loss.item()

#training model with one_hot with cross-entropy loss
def train_model_one_hot_cross_entropy(xs, ys, w, reg_strength,epochs, lr):
    w = intial_weights.clone().detach().requires_grad_(True)
    for epoch in range(epochs):
        # Forward pass
        xenc = F.one_hot(xs, num_classes=27*27).float()
        logits = xenc @ w
        probs = F.softmax(logits, dim=1)
        loss = F.cross_entropy(logits, ys)
        # Regularization
        reg_loss = reg_strength * (w**2).mean()
        total_loss = loss + reg_loss
        # Backward pass
        w.grad = None  # set to 0 the gradient
        total_loss.backward()

        # Update
        w.data += -lr * w.grad
    return w,total_loss.item()

# training model with indexing into rows of w,softmax loss
def train_model_indexing_softmax(xs, ys, w, reg_strength,epochs, lr):
  w = intial_weights.clone().detach().requires_grad_(True)
  for epoch in range(epochs):
    #Forward pass
    logits = w[xs]
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(len(ys)), ys].log().mean()
    #Regularization
    reg_loss = reg_strength * (w ** 2).mean()
    total_loss = loss + reg_loss
    #Backward pass
    w.grad = None  # set to 0 the gradient
    total_loss.backward()
    #Update
    w.data += -lr * w.grad
  return w,total_loss.item()

# training model with indexing into rows of w,cross_entropy loss
def train_model_indexing_cross_entropy(xs, ys, w, reg_strength,epochs, lr):
  w = intial_weights.clone().detach().requires_grad_(True)
  for epoch in range(epochs):
    #Forward pass
    logits = w[xs]
    probs = F.softmax(logits, dim=1)
    loss = F.cross_entropy(logits, ys)
    #Regularization
    reg_loss = reg_strength * (w ** 2).mean()
    total_loss = loss + reg_loss
    #Backward pass
    w.grad = None
    total_loss.backward()
    #Update
    w.data += -lr * w.grad
  return w,total_loss.item()


# new words generated based on one-hot,softmax loss
# use w=train_w_one_hot_softmax
def produce_words_one_hot_softmax(w, num_words=50):
  g = torch.Generator().manual_seed(2147483647)
  for i in range(num_words):
    out = []
    ix1 = 0
    ix2 = 0
    while True:
      idx = ix1 * 27 + ix2
      xenc = F.one_hot(torch.tensor([idx]), num_classes=27*27).float()
      logits = xenc @ w
      counts = logits.exp()
      probs = counts / counts.sum(1, keepdims=True)
      ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
      out.append(itos[ix])
      if ix == 0:
           break
      ix1 = ix2
      ix2 = ix
    print(''.join(out))

# new words generated based on one-hot,cross entropy loss
# use w=train_w_one_hot_cross_entropy
def produce_words_one_hot_cross_entropy(w, num_words=50):
  g = torch.Generator().manual_seed(2147483647)
  for i in range(num_words):
    out = []
    ix1 = 0
    ix2 = 0
    while True:
      idx = ix1 * 27 + ix2
      xenc = F.one_hot(torch.tensor([idx]), num_classes=27*27).float()
      logits = xenc @ w
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
      out.append(itos[ix])
      if ix == 0:
           break
      ix1 = ix2
      ix2 = ix
    print(''.join(out))

# new words generated based on indexing,softmax loss
# use w=train_w_indexing_softmax
def produce_words_indexing_softmax(w, num_words=50):
  g = torch.Generator().manual_seed(2147483647)
  for i in range(num_words):
    out = []
    ix1 = 0
    ix2 = 0
    while True:
      idx = ix1 * 27 + ix2
      logits = w[idx]
      counts = logits.exp()
      probs = counts / counts.sum(0, keepdims=True)
      ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
      out.append(itos[ix])
      if ix == 0:
           break
      ix1 = ix2
      ix2 = ix
    print(''.join(out))

# new words generated based on indexing,cross entropy loss
# use w=train_w_indexing_cross_entropy
def produce_words_indexing_cross_entropy(w, num_words=50):
  g = torch.Generator().manual_seed(2147483647)
  for i in range(num_words):
    out = []
    ix1 = 0
    ix2 = 0
    while True:
      idx = ix1 * 27 + ix2
      logits = w[idx]
      probs = F.softmax(logits, dim=0)
      ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
      out.append(itos[ix])
      if ix == 0:
           break
      ix1 = ix2
      ix2 = ix
    print(''.join(out))

# Evaluate one-hot,softmax model
def evaluate_one_hot_softmax(xs, ys, w):
    xenc = F.one_hot(xs, num_classes=27 * 27).float()
    logits = xenc @ w
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(len(ys)), ys].log().mean()
    return loss.item()

# Evaluate one-hot,cross-entropy model
def evaluate_one_hot_cross_entropy(xs, ys, w):
    xenc = F.one_hot(xs, num_classes=27 * 27).float()
    logits = xenc @ w
    probs = F.softmax(logits, dim=1)
    loss = F.cross_entropy(logits, ys)
    return loss.item()

# Evaluate indexing,softmax model
def evaluate_indexing_softmax(xs, ys, w):
    logits = w[xs]
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(len(ys)), ys].log().mean()
    return loss.item()

# Evaluate one-hot,cross-entropy model
def evaluate_indexing_cross_entropy(xs, ys, w):
    logits = w[xs]
    probs = F.softmax(logits, dim=1)
    loss = F.cross_entropy(logits, ys)
    return loss.item()


In [ ]:
train_w_one_hot_softmax,train_loss_one_hot_softmax = train_model_one_hot_softmax(train_xs, train_ys, w, reg_strength, epochs=50, lr=500)
print(f'{train_loss_one_hot_softmax=}')

train_loss_one_hot_softmax=2.18642258644104


In [ ]:
train_w_one_hot_cross_entropy,train_loss_one_hot_cross_entropy = train_model_one_hot_cross_entropy(train_xs, train_ys, w, reg_strength, epochs=50, lr=500)
print(f'{train_loss_one_hot_cross_entropy=}')

train_loss_one_hot_cross_entropy=2.1604061126708984


In [ ]:
train_w_indexing_softmax,train_loss_indexing_softmax = train_model_indexing_softmax(train_xs, train_ys, w, reg_strength, epochs=50, lr=500)
print(f'{train_loss_indexing_softmax=}')

2.1624019145965576


In [ ]:
train_w_indexing_cross_entropy,train_loss_indexing_cross_entropy = train_model_indexing_cross_entropy(train_xs, train_ys, w, reg_strength, epochs=50, lr=500)
print(f'{train_loss_indexing_cross_entropy=}')

2.1618425846099854


In [ ]:
dev_w_one_hot_softmax,dev_loss_one_hot_softmax = train_model_one_hot_softmax(dev_xs, dev_ys, w, reg_strength, epochs=50, lr=500)
print(f'{dev_loss_one_hot_softmax=}')

dev_loss_one_hot_softmax=tensor(2.0678, grad_fn=<AddBackward0>)


In [ ]:
dev_w_one_hot_cross_entropy,dev_loss_one_hot_cross_entropy = train_model_one_hot_cross_entropy(dev_xs, dev_ys, w, reg_strength, epochs=50, lr=500)
print(f'{dev_loss_one_hot_cross_entropy=}')

dev_loss_one_hot_cross_entropy=2.0745949745178223


In [ ]:
dev_w_indexing_softmax,dev_loss_indexing_softmax = train_model_indexing_softmax(dev_xs, dev_ys, w, reg_strength, epochs=50, lr=500)
print(f'{dev_loss_indexing_softmax=}')

In [ ]:
dev_w_indexing_cross_entropy,dev_loss_indexing_cross_entropy = train_model_indexing_cross_entropy(dev_xs, dev_ys, w, reg_strength, epochs=50, lr=500)
print(f'{dev_loss_indexing_cross_entropy=}')

dev_loss_indexing_cross_entropy=2.085721969604492


In [ ]:

# Regularization in one-hot ,softmax
regularization_strengths = [0, 0.1, 0.01, 0.001]
best_dev_loss_one_hot_softmax = float('inf')
best_w_one_hot_softmax = None
best_reg_strength_one_hot_softmax = None
print('One-hot, softmax model')
for reg_strength in regularization_strengths:
  train_w_one_hot_softmax,train_loss_one_hot_softmax = train_model_one_hot_softmax(train_xs, train_ys, w, reg_strength, epochs=50, lr=500)
  dev_w_one_hot_softmax,dev_loss_one_hot_softmax = train_model_one_hot_softmax(dev_xs, dev_ys, w, reg_strength, epochs=50, lr=500)
  print(f'Regularization Strength: {reg_strength}')
  print(f'Train Loss: {train_loss_one_hot_softmax}')
  print(f'Dev Loss: {dev_loss_one_hot_softmax}')
  if dev_loss_one_hot_softmax < best_dev_loss_one_hot_softmax:
    best_dev_loss_one_hot_softmax = dev_loss_one_hot_softmax
    best_w_one_hot_softmax = train_w_one_hot_softmax
    best_reg_strength_one_hot_softmax = reg_strength
print(f'{best_reg_strength_one_hot_softmax=}')

test_loss_one_hot_softmax = evaluate_one_hot_softmax(test_xs, test_ys, best_w_one_hot_softmax)
print(f'{test_loss_one_hot_softmax=}')


One-hot, softmax model
Regularization Strength: 0
Train Loss: 2.161203384399414
Dev Loss: 2.068009376525879
Regularization Strength: 0.1
Train Loss: 2.2520437240600586
Dev Loss: 2.1716761589050293
Regularization Strength: 0.01
Train Loss: 2.190692901611328
Dev Loss: 2.0814783573150635
Regularization Strength: 0.001
Train Loss: 2.161137104034424
Dev Loss: 2.069394826889038
best_reg_strength_one_hot_softmax=0
test_loss_one_hot_softmax=2.18503475189209


In [ ]:
# Regularization in one-hot ,cross entropy
regularization_strengths = [0, 0.1, 0.01, 0.001]
best_dev_loss_one_hot_cross_entropy = float('inf')
best_w_one_hot_cross_entropy = None
best_reg_strength_one_hot_cross_entropy = None
print('One-hot, cross-entropy model')
for reg_strength in regularization_strengths:
  train_w_one_hot_cross_entropy,train_loss_one_hot_cross_entropy = train_model_one_hot_cross_entropy(train_xs, train_ys, w, reg_strength, epochs=50, lr=500)
  dev_w_one_hot_cross_entropy,dev_loss_one_hot_cross_entropy = train_model_one_hot_cross_entropy(dev_xs, dev_ys, w, reg_strength, epochs=50, lr=500)
  print(f'Regularization Strength: {reg_strength}')
  print(f'Train Loss: {train_loss_one_hot_cross_entropy}')
  print(f'Dev Loss: {dev_loss_one_hot_cross_entropy}')
  if dev_loss_one_hot_cross_entropy < best_dev_loss_one_hot_cross_entropy:
    best_dev_loss_one_hot_cross_entropy = dev_loss_one_hot_cross_entropy
    best_w_one_hot_cross_entropy = train_w_one_hot_cross_entropy
    best_reg_strength_one_hot_cross_entropy = reg_strength
print(f'{best_reg_strength_one_hot_cross_entropy=}')

test_loss_one_hot_cross_entropy = evaluate_one_hot_cross_entropy(test_xs, test_ys, best_w_one_hot_cross_entropy)
print(f'{test_loss_one_hot_cross_entropy=}')

One-hot, cross-entropy model
Regularization Strength: 0
Train Loss: 2.163349151611328
Dev Loss: 2.0680091381073
Regularization Strength: 0.1
Train Loss: 2.2522454261779785
Dev Loss: 2.17167592048645
Regularization Strength: 0.01
Train Loss: 2.1902639865875244
Dev Loss: 2.0814781188964844
Regularization Strength: 0.001
Train Loss: 2.161142110824585
Dev Loss: 2.069394826889038
best_reg_strength_one_hot_cross_entropy=0
test_loss_one_hot_cross_entropy=2.18947172164917


In [ ]:

# Regularization in indexing ,softmax
regularization_strengths = [0, 0.1, 0.01, 0.001]
best_dev_loss_indexing_softmax = float('inf')
best_w_indexing_softmax = None
best_reg_strength_indexing_softmax = None
print('Indexing, softmax model')
for reg_strength in regularization_strengths:
  train_w_indexing_softmax,train_loss_indexing_softmax = train_model_indexing_softmax(train_xs, train_ys, w, reg_strength, epochs=50, lr=500)
  dev_w_indexing_softmax,dev_loss_indexing_softmax = train_model_indexing_softmax(dev_xs, dev_ys, w, reg_strength, epochs=50, lr=500)
  print(f'Regularization Strength: {reg_strength}')
  print(f'Train Loss: {train_loss_indexing_softmax}')
  print(f'Dev Loss: {dev_loss_indexing_softmax}')
  if dev_loss_indexing_softmax < best_dev_loss_indexing_softmax:
    best_dev_loss_indexing_softmax = dev_loss_indexing_softmax
    best_w_indexing_softmax = train_w_indexing_softmax
    best_reg_strength_indexing_softmax = reg_strength
print(f'{best_reg_strength_indexing_softmax=}')

test_loss_indexing_softmax = evaluate_indexing_softmax(test_xs, test_ys, best_w_indexing_softmax)
print(f'{test_loss_indexing_softmax=}')

Indexing, softmax model
Regularization Strength: 0
Train Loss: 2.1612956523895264
Dev Loss: 2.068009376525879
Regularization Strength: 0.1
Train Loss: 2.2549948692321777
Dev Loss: 2.1716761589050293
Regularization Strength: 0.01
Train Loss: 2.177875280380249
Dev Loss: 2.0814783573150635
Regularization Strength: 0.001
Train Loss: 2.1687848567962646
Dev Loss: 2.069394826889038
0
test_loss_indexing_softmax=2.1788523197174072


In [ ]:

# Regularization in indexing,cross-entropy
regularization_strengths = [0, 0.1, 0.01, 0.001]
best_dev_loss_indexing_cross_entropy = float('inf')
best_w_indexing_cross_entropy = None
best_reg_strength_indexing_cross_entropy = None
print('Indexing, cross-entropy model')
for reg_strength in regularization_strengths:
  train_w_indexing_cross_entropy,train_loss_indexing_cross_entropy = train_model_indexing_cross_entropy(train_xs, train_ys, w, reg_strength, epochs=50, lr=500)
  dev_w_indexing_cross_entropy,dev_loss_indexing_cross_entropy = train_model_indexing_cross_entropy(dev_xs, dev_ys, w, reg_strength, epochs=50, lr=500)
  print(f'Regularization Strength: {reg_strength}')
  print(f'Train Loss: {train_loss_indexing_cross_entropy}')
  print(f'Dev Loss: {dev_loss_indexing_cross_entropy}')
  if dev_loss_indexing_cross_entropy < best_dev_loss_indexing_cross_entropy:
    best_dev_loss_indexing_cross_entropy = dev_loss_indexing_cross_entropy
    best_w_indexing_cross_entropy = train_w_indexing_cross_entropy
    best_reg_strength_indexing_cross_entropy = reg_strength
print(f'{best_reg_strength_indexing_cross_entropy}')

test_loss_indexing_cross_entropy = evaluate_indexing_cross_entropy(test_xs, test_ys, best_w_indexing_cross_entropy)
print(f'{test_loss_indexing_cross_entropy=}')

Indexing, cross-entropy model
Regularization Strength: 0
Train Loss: 2.1612956523895264
Dev Loss: 2.0680091381073
Regularization Strength: 0.1
Train Loss: 2.254995346069336
Dev Loss: 2.1716761589050293
Regularization Strength: 0.01
Train Loss: 2.1778059005737305
Dev Loss: 2.0814783573150635
Regularization Strength: 0.001
Train Loss: 2.1676249504089355
Dev Loss: 2.069394826889038
0
test_loss_indexing_cross_entropy=2.178852081298828


In [ ]:
produce_words_one_hot_softmax(best_w_one_hot_softmax)

ouwade.
.
osaqah.
prelay.
ocnzi.
orin.
tonia.
oster.
yulinaaryanileniassibduinrwibel.
se.
yinayla.
te.
yarmumtrifoe.
umj.
phyashiah.
zaylicora.
ya.
zocken.
yandreerimouim.
yfvn.
thaasnhmvfjfopszxhxdgorfmxtyn.
il.
.
ree.
olany.
tin.
yosli.
isamey.
yumjlkujcmkhaubwyhepharccwarriyan.
na.
zita.
patyrabel.
ha.
ivellarocbzzhqmfwveawath.
.
.
odxsktqvslettkczsalee.
prin.
noewkflbjtnellianten.
ya.
yeusfso.
yodcdpqajbaliypjrgia.
tezrwaylia.
oywhqelvin.
osa.
omihaurik.
tim.
.
ossabdon.
oli.


In [ ]:
produce_words_one_hot_cross_entropy(best_w_one_hot_cross_entropy)

ouwade.
.
osaqah.
prelay.
ocnzi.
orin.
tonia.
oster.
yulinaaryanileniassibduinrwibel.
se.
yinayla.
te.
yarmumtrifoe.
umj.
phyashiah.
zaylicora.
ya.
zocken.
yandreerimouim.
yfvn.
thaasnhmvfjfopszxhxdgorfmxtyn.
il.
.
ree.
olan.
ot.
ryn.
zai.
isamey.
yumjlkujcmkhaubwyhepharccwarriyan.
na.
zita.
patyrabel.
ha.
ivellarocbzzhqmfwveawath.
.
.
odxsktqvslettkczsalee.
prin.
noewkflbjtnellianten.
ya.
yeusfso.
yodcdpqajbaliypjrgia.
tezrwaylia.
oywhqelvin.
osa.
omihaurik.
tim.
.
ossabdon.


In [ ]:
produce_words_indexing_softmax(best_w_indexing_softmax)

ouwade.
.
osaqah.
prelay.
ocnzi.
orin.
tonia.
oster.
yulinaaryanileniassibduinrwibel.
se.
yinayla.
te.
yarmumtrifoe.
umj.
phyashiah.
zaylicora.
ya.
zocken.
yandreerimouim.
yfvn.
thaasnhmvfjfopszxhxdgorfmxtyn.
il.
.
ree.
olany.
tin.
yosli.
isamey.
yumjlkujcmkhaubwyhepharccwarriyanhwannita.
patyrabel.
ha.
ivellarocbzzhqmfwveawath.
.
.
odxsktqvslettkczsalee.
prin.
noewkflbjtnellianten.
ya.
yeusfso.
yodcdpqajbaliypjrgia.
tezrwaylia.
oywhqelvin.
osa.
omihaurik.
tim.
.
ossabdon.
oli.
na.
prettwbjdgbannyah.


In [ ]:
produce_words_indexing_cross_entropy(best_w_indexing_cross_entropy)

ouwade.
.
osaqah.
prelay.
ocnzi.
orin.
tonia.
oster.
yulinaaryanileniassibduinrwibel.
se.
yinayla.
te.
yarmumtrifoe.
umj.
phyashiah.
zaylicora.
ya.
zocken.
yandreerimouim.
yfvn.
thaasnhmvfjfopszxhxdgorfmxtyn.
il.
.
ree.
olany.
tin.
yosli.
isamey.
yumjlkujcmkhaubwyhepharccwarriyanhwannita.
patyrabel.
ha.
ivellarocbzzhqmfwveawath.
.
.
odxsktqvslettkczsalee.
prin.
noewkflbjtnellianten.
ya.
yeusfso.
yodcdpqajbaliypjrgia.
tezrwaylia.
oywhqelvin.
osa.
omihaurik.
tim.
.
ossabdon.
oli.
na.
prettwbjdgbannyah.
